# ОПИСАНИЕ
В первой части мы провели анализ данных и построение **коллаборативной** рекомендательной системы. 

Во второй части - **content base** рекомендательная система основанная на семантическом сходстве контента. 

В этой части мы рассмотрим поиск на основе близости векторов BERT текстового описания контента.




# ИМПОРТ



In [1]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

In [2]:
import os
import sys
import re
import time
import pandas as pd
import numpy as np
import pprint
import tempfile
from typing import Dict, Text

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Embedding, Flatten, Input, concatenate, Reshape
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import tensorflow_hub as hub
import tensorflow_text as text

from matplotlib import pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
PATH = '/content/drive/MyDrive/Colab Notebooks/HACKATHONS/Видеопереворот/Рекомендательная система/files/'

# Обезличеная база данных видеоконтента 
concurs_data = 'concurs_data.csv'

# ДАННЫЕ

## 500 самых частых слов по закону Ципфа

In [5]:
# 500 Самых частых слов по закону Ципфа
cipf_500 = [
  'и',
  'в',
  'не',
  'на',
  'я',
  'быть',
  'он',
  'с',
  'что',
  'а',
  'по',
  'это',
  'она',
  'этот',
  'к',
  'но',
  'они',
  'мы',
  'как',
  'из',
  'у',
  'который',
  'то',
  'за',
  'свой',
  'что',
  'весь',
  'год',
  'от',
  'так',
  'о',
  'для',
  'ты',
  'же',
  'все',
  'тот',
  'мочь',
  'вы',
  'человек',
  'такой',
  'его',
  'сказать',
  'только',
  'или',
  'ещё',
  'бы',
  'себя',
  'один',
  'как',
  'уже',
  'до',
  'время',
  'если',
  'сам',
  'когда',
  'другой',
  'вот',
  'говорить',
  'наш',
  'мой',
  'знать',
  'стать',
  'при',
  'чтобы',
  'дело',
  'жизнь',
  'кто',
  'первый',
  'очень',
  'два',
  'день',
  'её',
  'новый',
  'рука',
  'даже',
  'во',
  'со',
  'раз',
  'где',
  'там',
  'под',
  'можно',
  'ну',
  'какой',
  'после',
  'их',
  'работа',
  'без',
  'самый',
  'потом',
  'надо',
  'хотеть',
  'ли',
  'слово',
  'идти',
  'большой',
  'должен',
  'место',
  'иметь',
  'ничто',
  'то',
  'сейчас',
  'тут',
  'лицо',
  'каждый',
  'друг',
  'нет',
  'теперь',
  'ни',
  'глаз',
  'тоже',
  'тогда',
  'видеть',
  'вопрос',
  'через',
  'да',
  'здесь',
  'дом',
  'да',
  'потому',
  'сторона',
  'какой-то',
  'думать',
  'сделать',
  'страна',
  'жить',
  'чем',
  'мир',
  'об',
  'последний',
  'случай',
  'голова',
  'более',
  'делать',
  'что-то',
  'смотреть',
  'ребенок',
  'просто',
  'конечно',
  'сила',
  'российский',
  'конец',
  'перед',
  'несколько',
  'вид',
  'система',
  'всегда',
  'работать',
  'между',
  'три',
  'нет',
  'понять',
  'пойти',
  'часть',
  'спросить',
  'город',
  'дать',
  'также',
  'никто',
  'понимать',
  'получить',
  'отношение',
  'лишь',
  'второй',
  'именно',
  'ваш',
  'хотя',
  'ни',
  'сидеть',
  'над',
  'женщина',
  'оказаться',
  'русский',
  'один',
  'взять',
  'прийти',
  'являться',
  'деньги',
  'почему',
  'вдруг',
  'любить',
  'стоить',
  'почти',
  'земля',
  'общий',
  'ведь',
  'машина',
  'однако',
  'сразу',
  'хорошо',
  'вода',
  'отец',
  'высокий',
  'остаться',
  'выйти',
  'много',
  'проблема',
  'начать',
  'хороший',
  'час',
  'это',
  'сегодня',
  'право',
  'совсем',
  'нога',
  'считать',
  'главный',
  'решение',
  'увидеть',
  'дверь',
  'казаться',
  'образ',
  'писать',
  'история',
  'лучший',
  'власть',
  'закон',
  'все',
  'война',
  'бог',
  'голос',
  'найти',
  'поэтому',
  'стоять',
  'вообще',
  'тысяча',
  'больше',
  'вместе',
  'маленький',
  'книга',
  'некоторый',
  'решить',
  'любой',
  'возможность',
  'результат',
  'ночь',
  'стол',
  'никогда',
  'имя',
  'область',
  'молодой',
  'пройти',
  'например',
  'статья',
  'оно',
  'число',
  'компания',
  'про',
  'государственный',
  'полный',
  'принять',
  'народ',
  'никакой',
  'советский',
  'жена',
  'настоящий',
  'всякий',
  'группа',
  'развитие',
  'процесс',
  'суд',
  'давать',
  'ответить',
  'старый',
  'условие',
  'твой',
  'пока',
  'средство',
  'помнить',
  'начало',
  'ждать',
  'свет',
  'пора',
  'путь',
  'душа',
  'куда',
  'нужно',
  'разный',
  'нужный',
  'уровень',
  'иной',
  'форма',
  'связь',
  'уж',
  'минута',
  'кроме',
  'находиться',
  'опять',
  'многий',
  'белый',
  'собственный',
  'улица',
  'черный',
  'написать',
  'вечер',
  'снова',
  'основной',
  'качество',
  'мысль',
  'дорога',
  'мать',
  'действие',
  'месяц',
  'оставаться',
  'государство',
  'язык',
  'любовь',
  'взгляд',
  'мама',
  'играть',
  'далекий',
  'лежать',
  'нельзя',
  'век',
  'школа',
  'подумать',
  'уйти',
  'цель',
  'среди',
  'общество',
  'посмотреть',
  'деятельность',
  'организация',
  'кто-то',
  'вернуться',
  'президент',
  'комната',
  'порядок',
  'момент',
  'театр',
  'следовать',
  'читать',
  'письмо',
  'подобный',
  'следующий',
  'утро',
  'особенно',
  'помощь',
  'ситуация',
  'роль',
  'бывать',
  'ходить',
  'рубль',
  'начинать',
  'появиться',
  'смысл',
  'состояние',
  'называть',
  'рядом',
  'квартира',
  'назад',
  'равный',
  'из-за',
  'орган',
  'внимание',
  'тело',
  'труд',
  'прийтись',
  'хотеться',
  'сын',
  'мера',
  'пять',
  'смерть',
  'живой',
  'рынок',
  'программа',
  'задача',
  'предприятие',
  'известный',
  'окно',
  'вести',
  'совершенно',
  'военный',
  'разговор',
  'показать',
  'правительство',
  'важный',
  'семья',
  'великий',
  'производство',
  'простой',
  'значит',
  'третий',
  'сколько',
  'огромный',
  'давно',
  'политический',
  'информация',
  'действительно',
  'положение',
  'поставить',
  'бояться',
  'наконец',
  'центр',
  'происходить',
  'ответ',
  'муж',
  'автор',
  'все-таки',
  'стена',
  'существовать',
  'даже',
  'интерес',
  'становиться',
  'федерация',
  'правило',
  'оба',
  'часто',
  'московский',
  'управление',
  'слышать',
  'быстро',
  'смочь',
  'заметить',
  'как-то',
  'мужчина',
  'долго',
  'правда',
  'идея',
  'партия',
  'иногда',
  'использовать',
  'пытаться',
  'готовый',
  'чуть',
  'зачем',
  'представить',
  'чувствовать',
  'создать',
  'совет',
  'счет',
  'сердце',
  'движение',
  'вещь',
  'материал',
  'неделя',
  'чувство',
  'затем',
  'данный',
  'заниматься',
  'продолжать',
  'красный',
  'глава',
  'ко',
  'слушать',
  'наука',
  'узнать',
  'ряд',
  'газета',
  'причина',
  'против',
  'плечо',
  'современный',
  'цена',
  'план',
  'приехать',
  'речь',
  'четыре',
  'отвечать',
  'точка',
  'основа',
  'товарищ',
  'культура',
  'слишком',
  'рассказывать',
  'вполне',
  'далее',
  'рассказать',
  'данные',
  'представлять',
  'мнение',
  'социальный',
  'около',
  'документ',
  'институт',
  'ход',
  'брать',
  'забыть',
  'проект',
  'ранний',
  'встреча',
  'особый',
  'целый',
  'директор',
  'провести',
  'спать',
  'плохой',
  'может',
  'впрочем',
  'сильный',
  'наверное',
  'скорый',
  'ведь',
  'срок',
  'палец',
  'опыт',
  'помочь',
  'больше',
  'приходить',
  'служба'
]

## Датафрейм

In [6]:
df_concurs = pd.read_csv(PATH + concurs_data)

In [7]:
df_concurs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _key           100000 non-null  int64 
 1   channel_name   99963 non-null   object
 2   comment_count  100000 non-null  int64 
 3   description    95205 non-null   object
 4   dislike_count  100000 non-null  int64 
 5   duration       100000 non-null  int64 
 6   fulltitle      100000 non-null  object
 7   like_count     100000 non-null  int64 
 8   tags           100000 non-null  object
 9   upload_date    100000 non-null  int64 
 10  uploader_url   99998 non-null   object
 11  view_count     100000 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 9.2+ MB


In [8]:
df_concurs.head()

,_key,channel_name,comment_count,description,dislike_count,duration,fulltitle,like_count,tags,upload_date,uploader_url,view_count
0,21350050,ZO'R TV,135,Официальный сайт: http://www.zortv.uz\nZO'RTV ...,0,1742,Bitta savol bor 1- son (10.07.2018),437,"[""zo'r tv"" 'zor tv' 'зур тв' 'зор тв']",1531170000,http://www.youtube.com/channel/UCisKdSbTsPt16y...,35735
1,71990270,Мир Белогорья,0,ЧП произошло в Краснояружском Староселье. Напо...,0,36,Супружеская пара подорвалась на боеприпасе неу...,23,['белгород' 'белгородская область' 'белгород н...,1669064400,http://www.youtube.com/@mirbelogor,931
2,61570019,Спорт-Экспресс,45,"Российский лыжник рассказал, каково чувствоать...",0,387,"Сергей Устюгов: «На горе может выключить так, ...",644,[],1578085200,http://www.youtube.com/user/sportexpress,30898
3,22003727,Extra News,0,Министр иностранных дел Венгрии Петер Сийярто ...,0,155,Венгрия удостоила Мантурова высшей госнаграды ...,1,['Новости' 'События' 'политика' 'народы'],1625691600,http://www.youtube.com/channel/UCYasXPsHhTVsn5...,63
4,28432077,Твой Дом - Тюрьма,0,Обязательно подпишитесь на наш канал! Воры в з...,0,220,"Коронован ""Дедом Хасаном"", воевал с Монголом. ...",63,['воры в законе 2019' 'криминал' 'киллеры' 'бл...,1560805200,http://www.youtube.com/user/max17111,18455


## Отберём популярные видео 
Не будем обрабатывать все видео - колаб(бесплатная версия) вылетает, не хватает ресурсов. В этом ноутбуке мы продемонстируем концепт, для возьмём то количество данных при которых при генерации векторов не вылетает программа 

In [9]:
df_popular = df_concurs.sort_values(by='view_count', ascending=False)[0:150]

In [10]:
df_popular['fulltitle'].head(30)

79153    Маша и Медведь (Masha and The Bear) - Маша плю...
69863    Маша и Медведь - Спи, моя радость, усни! (Сери...
12905    RC Cars MUD OFF Road — Land Rover Defender 90 ...
10902    RC Cars 4x4 Sands Storm Racing and MUD Action ...
69832    Маша и Медведь - Следы невиданных зверей🐾 (Сер...
8377       Маша и Медведь - Нынче все наоборот🔁 (Серия 38)
51444    Маша и Медведь (Masha and The Bear) - Следы не...
27094    Маша и Медведь - Песня “Сладкая жизнь” (Сладка...
42287    GAYAZOV$ BROTHER$ - Увезите меня на Дип-хаус |...
38017                             HammAli & Navai - Птичка
8083     Маша и Медведь - "Новогодняя песенка" (Раз, дв...
11695    Мультики про машинки:Грузовичок Лева - Все сер...
31452    Granny стала огромной! Вызываем Гренни! Granny...
99655    Фиксики - Нолик – пожарный: раз, два, три – ог...
55239    Три кота | Варенье в подвале | Серия 5 | Мульт...
52573    Щенячий патруль | Лучшие зимние приключения! ❄...
54458    Игра бродилка - Котёнок Котэ и Яйца - Мультик .

## Загрузка модели BERT
Мы загрузим две модели, одну для предварительной обработки, а другую для кодирования. Ссылки на модели приведены ниже.

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=True)

## Инициализация слоев BERT
В приведенном ниже коде мы создаем входной слой, используя tf.keras.layers.Input метод. Мы будем использовать в **preprocessed_text** качестве входных данных для этого слоя.

Затем **bert_encoder** функция преобразует предварительно обработанный текст в embadding векторы. Это будет выход этого слоя. Затем **outputs** они будут переданы в слои нейронной сети.

In [12]:
text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

## Проверим работу

In [13]:
p_text = bert_preprocess(['text'])
v = bert_encoder(p_text)

In [14]:
v['pooled_output'][0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([ 2.70360082e-01, -1.51426017e-01,  2.02479690e-01,  1.36259735e-01,
       -1.11354217e-01,  2.72155672e-01,  1.81260586e-01,  2.49676630e-01,
       -4.55122530e-01,  2.98245847e-01,  2.79911589e-02, -6.94658905e-02,
       -1.89139619e-01, -1.81658924e-01,  3.23850587e-02, -1.17484927e-01,
        4.35243487e-01,  8.09016544e-03, -1.77940235e-01, -4.19278741e-01,
       -9.99817014e-01, -2.44627044e-01, -3.81946921e-01, -2.06391916e-01,
       -3.17560732e-01,  1.76368654e-01, -6.06947541e-02,  2.72821367e-01,
        3.63530397e-01, -1.30347505e-01,  3.64892185e-04, -9.99835253e-01,
        4.42686290e-01,  5.91138065e-01,  2.33706027e-01, -8.11598971e-02,
        1.85111642e-01,  1.85093209e-01,  3.22861195e-01, -3.63657027e-01,
        2.37809997e-02, -1.49478361e-01,  1.35586590e-01,  1.46379545e-01,
        1.47330984e-01, -3.42431307e-01, -8.86009410e-02,  6.14471436e-02,
       -3.10208708e-01,  5.89239858e-02, -1.73562448

## Создадим BERT вектора
В рабочей версии - вектор должен создаваться при загрузке видео, а не на этом шаге - для того, что бы сократить количество операций.

### Функция получения ngramm

In [15]:
time_start = time.time()

key_list = []
vector_list = []
for i in range(df_popular.shape[0]):
  df_txt = df_popular.iloc[i]

  key = df_txt['_key']

  '''
  if pd.notna(df_txt['description']):
    description = re.sub(r'[^\w\d\s\-]', '', df_txt['description'])
    description_c = re.sub(r'\s{2,}', ' ', description)
    if len(description_c) > 2:
      description_v = bert_encoder(bert_preprocess([description_c]))['pooled_output'][0]
      key_list.append([df_txt['_key'], df_txt['fulltitle'], df_txt['description']])
      vector_list.append(description_v)
  '''

  if pd.notna(df_txt['fulltitle']):
    # Разбираем на предложения
    fulltitle_arr = sent_tokenize(df_txt['fulltitle'])
  
    for sent in fulltitle_arr:
      sent = re.sub(r'[^\w\d\s\-]', '', sent)
      sent = re.sub(r'\s{2,}', ' ', sent)
      if len(sent) > 2:
        # Добавляем в вектор всё предложение
        vector = bert_encoder(bert_preprocess([sent]))['pooled_output'][0]
        key_list.append([df_txt['_key'], df_txt['fulltitle'], df_txt['description']])
        vector_list.append(vector)

        # Создаём биграммы
        words_list = sent.split()
        if (len(words_list) >= 2):
          ngrm = ngrams(words_list, 2)
          for ng in ngrm:
            # Если первое или последнее слово в списке self.cipf_500 - пропускаем n-грамму
            if ng[0] in cipf_500 or ng[-1] in cipf_500:
              continue
            s = ' '.join(ng).strip()
            s = re.sub(r'[^\w\d\s]', '', s)
            # Если в n-грамме нет слова (от 3х букв) - пропускаем n-грамму
            if re.search(r'[^a-zA-zа-яёА-ЯЁ]{3,}', s):
              continue

            vector = bert_encoder(bert_preprocess([s]))['pooled_output'][0]
            key_list.append([df_txt['_key'], df_txt['fulltitle'], df_txt['description']])
            vector_list.append(vector)

        # Создаём триграммы
        words_list = sent.split()
        if (len(words_list) >= 3):
          ngrm = ngrams(words_list, 3)
          for ng in ngrm:
            # Если первое или последнее слово в списке self.cipf_500 - пропускаем n-грамму
            if ng[0] in cipf_500 or ng[-1] in cipf_500:
              continue
            s = ' '.join(ng).strip()
            s = re.sub(r'[^\w\d\s]', '', s)
            # Если в n-грамме нет слова (от 3х букв) - пропускаем n-грамму
            if re.search(r'[^a-zA-zа-яёА-ЯЁ]{3,}', s):
              continue

            vector = bert_encoder(bert_preprocess([s]))['pooled_output'][0]
            key_list.append([df_txt['_key'], df_txt['fulltitle'], df_txt['description']])
            vector_list.append(vector)

time_delta = round(time.time() - time_start)
# print(f'Время выполнения: {time_delta}c')

In [16]:
df_key = pd.DataFrame(key_list, columns=['key', 'fulltitle', 'description'])
df_key.shape

(1685, 3)

## Ближайшие соседи

In [17]:
neigh = NearestNeighbors(n_neighbors=30, n_jobs=-1, metric='cosine') 
neigh.fit(vector_list)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=30)

## Протестируем

### Функции

In [18]:
# Ближайшие соседи
def get_search(df_key, neigh, text):
  # Получим вектор для текста
  text = re.sub(r'[^\w\d\s\-]', '', text)
  text = re.sub(r'\s{2,}', ' ', text)
  vector = bert_encoder(bert_preprocess([text]))['pooled_output'][0]

  # Получим ближайших соседей
  res = neigh.kneighbors([vector], return_distance=True)
  idx = res[1][0]
  nearest = df_key.iloc[idx]
  return nearest['fulltitle'].unique()[0:10]

### Тесты

In [19]:
text = "Маша и медведь"
search = get_search(df_key, neigh, text)
print(f'ИСКОМАЯ ФРАЗА: "{text}"')
print()
print('РЕЗУЛЬТАТ:')
for i in range(10):
  print(search[i])

ИСКОМАЯ ФРАЗА: "Маша и медведь"

РЕЗУЛЬТАТ:
Маша и Медведь - Спи, моя радость, усни! (Серия 62)
Маша и Медведь - Нынче все наоборот🔁 (Серия 38)
Маша и Медведь - Песня “Сладкая жизнь” (Сладкая жизнь)
Маша и Медведь - 🤡 Детский Праздник! 👧🏻👶🧒
Маша и Медведь 💥 НОВАЯ СЕРИЯ! 💥Кушать подано😋🪴 Коллекция мультиков для детей про Машу
Маша и Медведь - Следы невиданных зверей🐾 (Серия 4)
Маша и Медведь - "Новогодняя песенка" (Раз, два, три! Елочка, гори!)
Маша и Медведь. Ловись рыбка. Какие замечательные червячки!
Маша и Медведь (Masha and The Bear) - Маша плюс каша (17 Серия)
Маша и Медведь (Masha and The Bear) - Следы невиданных зверей (4 Серия)


In [20]:
text = "Щенячий патруль"
search = get_search(df_key, neigh, text)
print(f'ИСКОМАЯ ФРАЗА: "{text}"')
print()
print('РЕЗУЛЬТАТ:')
for i in range(10):
  print(search[i])

ИСКОМАЯ ФРАЗА: "Щенячий патруль"

РЕЗУЛЬТАТ:
Щенячий патруль | Лучшие зимние приключения! ❄️ | Nick Jr. Россия
Щенячий патруль | Пасха уже тут | Nick Jr. Россия
Проблемный робот (VR)
Учим алфавит - Синий трактор - Песенки мультики для детей малышей
РАКЕТА - Синий трактор - Развивающий мультик песенка для детей малышей про космос планеты и звёзды
Помочь раненому Геккону | Суперспасатели | Малыш панда | Kids cartoon | BabyBus
Леденцы на Палочке из Шариков  мультик для детей #ВолшебствоТВ
Говорящий Том и Друзья, 20 серия - Крепость из одеял | Мультики для детей
ФИКСИКИ: мультик из игрушек. Селфи палка
Злая соверма делает кусь. Сыч Юйня в платочке


In [21]:
text = "Детские мультфильмы"
search = get_search(df_key, neigh, text)
print(f'ИСКОМАЯ ФРАЗА: "{text}"')
print()
print('РЕЗУЛЬТАТ:')
for i in range(10):
  print(search[i])

ИСКОМАЯ ФРАЗА: "Детские мультфильмы"

РЕЗУЛЬТАТ:
Три кота | За чайником | Серия 95 | Мультфильмы для детей
Три кота | Потерянный динозавр | Новая Серия 121 | Мультфильмы для детей
Три кота | Варенье в подвале | Серия 5 | Мультфильмы для детей
Жихарка | русский мультфильм | дети видео | мультфильмы | Zhikharka | Moral Stories | Kids Cartoon
Малышарики - Магнитик - серия 50 - обучающие мультфильмы для малышей 0-4
▶️ Во имя любви - Мелодрама | Фильмы и сериалы - Русские мелодрамы
Охраняем сон БЕБИ БОН! – Весёлые игры Как Мама с Baby Born. Смешные видео куклы для девочек.
Буба и неуловимый Мяч ⚽ Смешной мультфильм ⚽ Классные Мультики
Чуддики | В Кинотеатре ... | Смешные мультики для детей
Буба - Гость - Серия 51 - Мультфильм для детей
